In [3]:
# =============================================================================
# 🌌 GALACTIC EXOPLANET DISCOVERY & HABITABILITY ASSESSMENT SYSTEM (GEDHAS)
# =============================================================================
#
# EXECUTIVE SUMMARY:
# ------------------
# This interactive database application simulates a comprehensive exoplanet
# discovery and analysis platform. It models how astronomers catalog newly
# discovered exoplanets and assess their potential for harboring life.
#
# The system tracks:
# - Star systems with their physical properties (luminosity, temperature, age)
# - Exoplanets with orbital and physical characteristics
# - Discovery missions and their detection methods
# - Habitability assessments using the Earth Similarity Index (ESI)
# - Atmospheric composition analysis
#
# SCHEMA DESIGN (Entity-Relationship Description):
# -----------------------------------------------
# 1.  STAR_SYSTEMS: Parent table containing host star information
#    - star_id (PK), name, spectral_type, luminosity, temperature, distance, age
#
# 2.  EXOPLANETS: Child table linked to stars (many planets per star possible)
#    - planet_id (PK), star_id (FK), name, mass, radius, orbital_period,
#      semi_major_axis, eccentricity, discovery_date, equilibrium_temp
#
# 3. DISCOVERY_MISSIONS: Tracks space telescopes and ground observatories
#    - mission_id (PK), name, launch_date, detection_method, status
#
# 4. PLANET_DISCOVERIES: Junction table linking planets to discovery missions
#    - discovery_id (PK), planet_id (FK), mission_id (FK), discovery_date,
#      confirmation_status, discovery_team
#
# 5. ATMOSPHERIC_ANALYSIS: Stores atmospheric composition data
#    - analysis_id (PK), planet_id (FK), hydrogen_pct, helium_pct, nitrogen_pct,
#      oxygen_pct, co2_pct, methane_pct, water_vapor_pct, biosignature_detected
#
# 6. HABITABILITY_SCORES: Calculated habitability metrics
#    - score_id (PK), planet_id (FK), esi_score, hz_status, water_probability,
#      atmosphere_rating, overall_habitability_class
#
# RELATIONSHIPS:
# - STAR_SYSTEMS (1) → (Many) EXOPLANETS
# - EXOPLANETS (1) → (Many) PLANET_DISCOVERIES
# - DISCOVERY_MISSIONS (1) → (Many) PLANET_DISCOVERIES
# - EXOPLANETS (1) → (1) ATMOSPHERIC_ANALYSIS
# - EXOPLANETS (1) → (1) HABITABILITY_SCORES
#
# Author: Database Systems Coursework
# Date: November 2025
# =============================================================================


# # 🌌 GEDHAS: Galactic Exoplanet Discovery & Habitability Assessment System
#
# ## An Interactive Database Dashboard for Exoplanet Analysis
#
# This notebook implements a sophisticated database system for cataloging and analyzing
# exoplanets discovered across our galaxy. It demonstrates advanced SQL techniques,
# complex data relationships, and interactive visualizations that would be used by
# astronomers and astrobiologists to identify potentially habitable worlds.

# %%
# =============================================================================
# SECTION 1: IMPORTS AND CONFIGURATION
# =============================================================================
# Import all necessary libraries for database operations, data generation,
# and interactive visualizations

import sqlite3                          # SQLite database interface
import random                           # Random number generation for mock data
import math                             # Mathematical functions for astronomical calculations
from datetime import datetime, timedelta  # Date handling for discovery dates
import numpy as np                      # Numerical operations
import pandas as pd                     # Data manipulation and analysis

# Visualization libraries
import matplotlib. pyplot as plt         # Static plotting
import matplotlib.patches as mpatches   # For custom legend patches
from matplotlib.colors import LinearSegmentedColormap  # Custom colormaps

# Try to import interactive widgets (for Colab compatibility)
try:
    import ipywidgets as widgets        # Interactive UI widgets
    from IPython.display import display, HTML, clear_output
    WIDGETS_AVAILABLE = True
except ImportError:
    WIDGETS_AVAILABLE = False
    print("Note: ipywidgets not available.  Using static visualizations.")

# Try to import plotly for advanced interactive charts
try:
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    PLOTLY_AVAILABLE = True
except ImportError:
    PLOTLY_AVAILABLE = False
    print("Note: Plotly not available.  Using matplotlib for all charts.")

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Configure matplotlib for better visuals
plt.style.use('dark_background')  # Space theme!
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 10

print("✅ All imports successful!")
print(f"   - Interactive widgets: {'Available' if WIDGETS_AVAILABLE else 'Not available'}")
print(f"   - Plotly charts: {'Available' if PLOTLY_AVAILABLE else 'Not available'}")

# %%
# =============================================================================
# SECTION 2: DATABASE SCHEMA CREATION (DDL - Data Definition Language)
# =============================================================================
# This section creates the complete database schema with 6 interrelated tables
# modeling the exoplanet discovery and analysis domain

def create_database():
    """
    Creates the GEDHAS SQLite database with all required tables.

    Returns:
        sqlite3.Connection: Database connection object

    Schema includes:
        - STAR_SYSTEMS: Host star properties
        - EXOPLANETS: Planet physical and orbital characteristics
        - DISCOVERY_MISSIONS: Space telescopes and observatories
        - PLANET_DISCOVERIES: Links planets to their discovery missions
        - ATMOSPHERIC_ANALYSIS: Atmospheric composition data
        - HABITABILITY_SCORES: Calculated habitability metrics
    """

    # Create an in-memory database (faster for demonstration)
    # In production, you would use a file: sqlite3.connect('gedhas.db')
    conn = sqlite3.connect(':memory:')
    cursor = conn.cursor()

    print("🔧 Creating GEDHAS Database Schema...")
    print("=" * 60)

    # -------------------------------------------------------------------------
    # Table 1: STAR_SYSTEMS
    # Stores information about host stars where exoplanets have been discovered
    # -------------------------------------------------------------------------
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS STAR_SYSTEMS (
            star_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL UNIQUE,
            -- Spectral classification (O,B,A,F,G,K,M) determines star properties
            spectral_type TEXT NOT NULL,
            -- Luminosity relative to our Sun (L☉)
            luminosity REAL NOT NULL,
            -- Surface temperature in Kelvin
            temperature INTEGER NOT NULL,
            -- Distance from Earth in light-years
            distance_ly REAL NOT NULL,
            -- Age in billions of years
            age_gyr REAL NOT NULL,
            -- Mass relative to our Sun (M☉)
            mass_solar REAL NOT NULL,
            -- Right Ascension coordinate (degrees)
            ra_deg REAL NOT NULL,
            -- Declination coordinate (degrees)
            dec_deg REAL NOT NULL,
            -- Galactic quadrant for regional analysis
            galactic_quadrant TEXT NOT NULL,
            -- Metallicity ([Fe/H]) affects planet formation likelihood
            metallicity REAL NOT NULL
        )
    ''')
    print("   ✓ Created STAR_SYSTEMS table")

    # -------------------------------------------------------------------------
    # Table 2: EXOPLANETS
    # Core table storing all discovered exoplanets with physical properties
    # -------------------------------------------------------------------------
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS EXOPLANETS (
            planet_id INTEGER PRIMARY KEY AUTOINCREMENT,
            -- Foreign key linking to host star
            star_id INTEGER NOT NULL,
            name TEXT NOT NULL UNIQUE,
            -- Mass relative to Earth (M⊕)
            mass_earth REAL NOT NULL,
            -- Radius relative to Earth (R⊕)
            radius_earth REAL NOT NULL,
            -- Orbital period in Earth days
            orbital_period_days REAL NOT NULL,
            -- Semi-major axis in AU (Astronomical Units)
            semi_major_axis_au REAL NOT NULL,
            -- Orbital eccentricity (0 = circular, closer to 1 = elliptical)
            eccentricity REAL NOT NULL,
            -- Equilibrium temperature in Kelvin
            equilibrium_temp_k REAL NOT NULL,
            -- Surface gravity relative to Earth
            surface_gravity_earth REAL NOT NULL,
            -- Density in g/cm³
            density_gcc REAL NOT NULL,
            -- Planet type classification
            planet_type TEXT NOT NULL,
            -- Date of initial detection
            discovery_date DATE NOT NULL,
            -- Confirmation status
            confirmed BOOLEAN NOT NULL DEFAULT 1,
            FOREIGN KEY (star_id) REFERENCES STAR_SYSTEMS(star_id)
        )
    ''')
    print("   ✓ Created EXOPLANETS table")

    # -------------------------------------------------------------------------
    # Table 3: DISCOVERY_MISSIONS
    # Catalogs space telescopes and ground observatories discovering planets
    # -------------------------------------------------------------------------
    cursor. execute('''
        CREATE TABLE IF NOT EXISTS DISCOVERY_MISSIONS (
            mission_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL UNIQUE,
            -- Organization operating the mission
            organization TEXT NOT NULL,
            -- Type: 'Space Telescope', 'Ground Observatory', 'Survey'
            mission_type TEXT NOT NULL,
            -- Detection method used
            detection_method TEXT NOT NULL,
            -- Mission launch/start date
            launch_date DATE NOT NULL,
            -- Mission end date (NULL if ongoing)
            end_date DATE,
            -- Current operational status
            status TEXT NOT NULL,
            -- Total confirmed discoveries by this mission
            total_discoveries INTEGER DEFAULT 0,
            -- Sensitivity rating (1-10)
            sensitivity_rating INTEGER NOT NULL
        )
    ''')
    print("   ✓ Created DISCOVERY_MISSIONS table")

    # -------------------------------------------------------------------------
    # Table 4: PLANET_DISCOVERIES
    # Junction table linking planets to discovery missions with metadata
    # -------------------------------------------------------------------------
    cursor. execute('''
        CREATE TABLE IF NOT EXISTS PLANET_DISCOVERIES (
            discovery_id INTEGER PRIMARY KEY AUTOINCREMENT,
            planet_id INTEGER NOT NULL,
            mission_id INTEGER NOT NULL,
            -- Date this mission detected the planet
            detection_date DATE NOT NULL,
            -- Role: 'Primary Discoverer', 'Confirmation', 'Follow-up'
            discovery_role TEXT NOT NULL,
            -- Signal-to-noise ratio of detection
            snr_value REAL NOT NULL,
            -- Number of transit events observed (for transit method)
            transit_events INTEGER,
            -- Research team/institution credited
            discovery_team TEXT NOT NULL,
            -- Publication reference
            publication_ref TEXT,
            FOREIGN KEY (planet_id) REFERENCES EXOPLANETS(planet_id),
            FOREIGN KEY (mission_id) REFERENCES DISCOVERY_MISSIONS(mission_id)
        )
    ''')
    print("   ✓ Created PLANET_DISCOVERIES table")

    # -------------------------------------------------------------------------
    # Table 5: ATMOSPHERIC_ANALYSIS
    # Stores spectroscopic analysis of planetary atmospheres
    # -------------------------------------------------------------------------
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS ATMOSPHERIC_ANALYSIS (
            analysis_id INTEGER PRIMARY KEY AUTOINCREMENT,
            planet_id INTEGER NOT NULL UNIQUE,
            -- Analysis date
            analysis_date DATE NOT NULL,
            -- Atmospheric composition percentages
            hydrogen_pct REAL DEFAULT 0,
            helium_pct REAL DEFAULT 0,
            nitrogen_pct REAL DEFAULT 0,
            oxygen_pct REAL DEFAULT 0,
            carbon_dioxide_pct REAL DEFAULT 0,
            methane_pct REAL DEFAULT 0,
            water_vapor_pct REAL DEFAULT 0,
            ammonia_pct REAL DEFAULT 0,
            argon_pct REAL DEFAULT 0,
            other_pct REAL DEFAULT 0,
            -- Surface pressure in Earth atmospheres
            surface_pressure_atm REAL,
            -- Presence of potential biosignatures
            biosignature_detected BOOLEAN DEFAULT 0,
            -- Atmospheric stability assessment
            atmosphere_stability TEXT,
            -- Greenhouse effect strength (1-10)
            greenhouse_rating INTEGER,
            FOREIGN KEY (planet_id) REFERENCES EXOPLANETS(planet_id)
        )
    ''')
    print("   ✓ Created ATMOSPHERIC_ANALYSIS table")

    # -------------------------------------------------------------------------
    # Table 6: HABITABILITY_SCORES
    # Calculated habitability metrics for each planet
    # -------------------------------------------------------------------------
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS HABITABILITY_SCORES (
            score_id INTEGER PRIMARY KEY AUTOINCREMENT,
            planet_id INTEGER NOT NULL UNIQUE,
            -- Earth Similarity Index (0-1, higher = more Earth-like)
            esi_score REAL NOT NULL,
            -- Habitable Zone status
            hz_status TEXT NOT NULL,
            -- Probability of liquid water (0-100%)
            water_probability REAL NOT NULL,
            -- Atmosphere habitability rating (1-10)
            atmosphere_rating INTEGER NOT NULL,
            -- Magnetic field probability (affects radiation protection)
            magnetic_field_probability REAL NOT NULL,
            -- Tidal locking probability
            tidal_lock_probability REAL NOT NULL,
            -- Overall habitability classification
            habitability_class TEXT NOT NULL,
            -- Priority ranking for future study (1 = highest)
            study_priority INTEGER NOT NULL,
            -- Last calculation timestamp
            calculated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (planet_id) REFERENCES EXOPLANETS(planet_id)
        )
    ''')
    print("   ✓ Created HABITABILITY_SCORES table")

    # -------------------------------------------------------------------------
    # Create indexes for performance optimization on frequently queried columns
    # -------------------------------------------------------------------------
    cursor. execute('CREATE INDEX idx_exoplanets_star ON EXOPLANETS(star_id)')
    cursor.execute('CREATE INDEX idx_exoplanets_type ON EXOPLANETS(planet_type)')
    cursor. execute('CREATE INDEX idx_discoveries_planet ON PLANET_DISCOVERIES(planet_id)')
    cursor. execute('CREATE INDEX idx_discoveries_mission ON PLANET_DISCOVERIES(mission_id)')
    cursor.execute('CREATE INDEX idx_habitability_class ON HABITABILITY_SCORES(habitability_class)')
    cursor.execute('CREATE INDEX idx_habitability_esi ON HABITABILITY_SCORES(esi_score)')
    print("   ✓ Created performance indexes")

    conn.commit()
    print("=" * 60)
    print("✅ Database schema created successfully!")

    return conn

# %%
# =============================================================================
# SECTION 3: ASTRONOMICAL CALCULATION FUNCTIONS
# =============================================================================
# These functions implement real astronomical formulas used in exoplanet science

def calculate_habitable_zone(luminosity, temp_k):
    """
    Calculates the inner and outer boundaries of a star's habitable zone.

    The habitable zone (HZ) is the region around a star where liquid water
    could exist on a planet's surface. This uses the Kopparapu et al. (2013)
    formulation.

    Args:
        luminosity: Star's luminosity relative to the Sun (L☉)
        temp_k: Star's effective temperature in Kelvin

    Returns:
        tuple: (inner_boundary_au, outer_boundary_au)

    Formula Reference:
        Kopparapu, R. K., et al. (2013). "Habitable Zones Around Main-Sequence Stars"
    """
    # Temperature offset from solar temperature (5780K)
    t_star = temp_k - 5780

    # Coefficients for conservative HZ boundaries
    # Inner edge: "Runaway Greenhouse" limit
    s_eff_inner = 1.0146 + 8.1884e-5 * t_star + 1.9394e-9 * t_star**2
    # Outer edge: "Maximum Greenhouse" limit
    s_eff_outer = 0.3507 + 5.9578e-5 * t_star + 1.6707e-9 * t_star**2

    # Calculate boundaries in AU
    inner_hz = math.sqrt(luminosity / s_eff_inner)
    outer_hz = math.sqrt(luminosity / s_eff_outer)

    return (inner_hz, outer_hz)


def calculate_esi(radius_earth, mass_earth, temp_k, escape_vel_ratio=1.0):
    """
    Calculates the Earth Similarity Index (ESI) for an exoplanet.

    ESI is a measure of how physically similar a planet is to Earth,
    ranging from 0 (completely different) to 1 (identical to Earth).

    Args:
        radius_earth: Planet radius relative to Earth
        mass_earth: Planet mass relative to Earth
        temp_k: Planet equilibrium temperature in Kelvin
        escape_vel_ratio: Escape velocity relative to Earth

    Returns:
        float: ESI score between 0 and 1

    Formula Reference:
        Schulze-Makuch, D., et al. (2011). "A Two-Tiered Approach to
        Assessing the Habitability of Exoplanets"
    """
    # Reference values (Earth)
    r_ref = 1.0      # Earth radius
    m_ref = 1.0      # Earth mass
    t_ref = 288.0    # Earth mean temp (K)
    v_ref = 1.0      # Earth escape velocity

    # Weight exponents for each parameter
    w_r = 0.57       # Radius weight
    w_m = 1.07       # Mass weight (density proxy)
    w_t = 5.58       # Temperature weight
    w_v = 0.70       # Escape velocity weight

    # Calculate individual similarity indices
    esi_r = (1 - abs((radius_earth - r_ref) / (radius_earth + r_ref))) ** w_r
    esi_m = (1 - abs((mass_earth - m_ref) / (mass_earth + m_ref))) ** w_m
    esi_t = (1 - abs((temp_k - t_ref) / (temp_k + t_ref))) ** w_t
    esi_v = (1 - abs((escape_vel_ratio - v_ref) / (escape_vel_ratio + v_ref))) ** w_v

    # Geometric mean of all indices
    esi = (esi_r * esi_m * esi_t * esi_v) ** 0.25

    return min(max(esi, 0), 1)  # Clamp between 0 and 1


def calculate_equilibrium_temperature(star_temp, star_radius, semi_major_axis, albedo=0.3):
    """
    Calculates the equilibrium temperature of a planet.

    This is the theoretical temperature a planet would have if it were
    a blackbody in thermal equilibrium with its star.

    Args:
        star_temp: Star's effective temperature in Kelvin
        star_radius: Star's radius in solar radii
        semi_major_axis: Planet's orbital distance in AU
        albedo: Planet's bond albedo (fraction of light reflected)

    Returns:
        float: Equilibrium temperature in Kelvin
    """
    # Convert to consistent units
    r_star_m = star_radius * 6.957e8  # Solar radius to meters
    a_m = semi_major_axis * 1.496e11  # AU to meters

    # Stefan-Boltzmann equilibrium
    t_eq = star_temp * ((1 - albedo) ** 0.25) * math.sqrt(r_star_m / (2 * a_m))

    return t_eq


def classify_planet_type(mass_earth, radius_earth, temp_k):
    """
    Classifies a planet into a category based on its physical properties.

    Args:
        mass_earth: Planet mass relative to Earth
        radius_earth: Planet radius relative to Earth
        temp_k: Equilibrium temperature in Kelvin

    Returns:
        str: Planet classification
    """
    density = mass_earth / (radius_earth ** 3)  # Relative density

    if mass_earth < 0.1:
        return "Dwarf Planet"
    elif mass_earth < 2 and radius_earth < 1.5:
        if 200 < temp_k < 350:
            return "Potentially Habitable Terrestrial"
        elif temp_k >= 350:
            return "Hot Terrestrial"
        else:
            return "Cold Terrestrial"
    elif mass_earth < 10 and radius_earth < 2.5:
        if temp_k >= 350:
            return "Hot Super-Earth"
        elif 200 < temp_k < 350:
            return "Temperate Super-Earth"
        else:
            return "Cold Super-Earth"
    elif mass_earth < 20 and radius_earth < 4:
        return "Mini-Neptune"
    elif mass_earth < 100:
        return "Neptune-like"
    elif mass_earth < 500:
        return "Sub-Jupiter"
    elif mass_earth < 3000:
        if temp_k > 1000:
            return "Hot Jupiter"
        else:
            return "Jupiter-like"
    else:
        return "Super-Jupiter"


def classify_habitability(esi_score, hz_status, water_prob, atm_rating):
    """
    Assigns a habitability classification based on multiple factors.

    Args:
        esi_score: Earth Similarity Index (0-1)
        hz_status: Habitable zone position
        water_prob: Probability of liquid water (0-100)
        atm_rating: Atmosphere habitability rating (1-10)

    Returns:
        str: Habitability class
    """
    # Calculate composite score
    hz_bonus = 1.0 if hz_status == "In HZ" else 0.5 if "Near" in hz_status else 0.2
    composite = (esi_score * 0.4 + (water_prob/100) * 0.3 +
                 (atm_rating/10) * 0.2 + hz_bonus * 0.1)

    if composite >= 0.8 and hz_status == "In HZ":
        return "Class I: Prime Candidate"
    elif composite >= 0.6 and "HZ" in hz_status:
        return "Class II: High Potential"
    elif composite >= 0.4:
        return "Class III: Moderate Interest"
    elif composite >= 0.2:
        return "Class IV: Low Priority"
    else:
        return "Class V: Non-Habitable"


print("✅ Astronomical calculation functions defined!")

# %%
# =============================================================================
# SECTION 4: BATCH DATA GENERATION
# =============================================================================
# Generate large volumes of realistic mock data using Python

def generate_star_systems(conn, num_stars=150):
    """
    Generates realistic mock data for star systems.

    Uses actual astronomical distributions for spectral types, temperatures,
    and luminosities based on the Hertzsprung-Russell diagram.

    Args:
        conn: SQLite database connection
        num_stars: Number of star systems to generate

    Returns:
        list: List of star_ids created
    """
    cursor = conn.cursor()

    # Spectral type distribution (based on stellar population in solar neighborhood)
    # Format: (type, probability, temp_range, luminosity_range, mass_range)
    spectral_data = [
        ('M', 0.45, (2400, 3700), (0.001, 0.08), (0.08, 0.45)),    # Red dwarfs (most common)
        ('K', 0.25, (3700, 5200), (0.08, 0.6), (0.45, 0.8)),        # Orange dwarfs
        ('G', 0.15, (5200, 6000), (0.6, 1.5), (0.8, 1.04)),         # Sun-like
        ('F', 0.08, (6000, 7500), (1.5, 5), (1.04, 1.4)),           # Yellow-white
        ('A', 0.04, (7500, 10000), (5, 25), (1.4, 2.1)),            # White
        ('B', 0.02, (10000, 30000), (25, 30000), (2.1, 16)),        # Blue-white
        ('O', 0.01, (30000, 50000), (30000, 1000000), (16, 150)),   # Blue (rare)
    ]

    # Greek letter designations for star naming
    greek_letters = ['Alpha', 'Beta', 'Gamma', 'Delta', 'Epsilon', 'Zeta',
                     'Eta', 'Theta', 'Iota', 'Kappa', 'Lambda', 'Mu',
                     'Nu', 'Xi', 'Omicron', 'Pi', 'Rho', 'Sigma', 'Tau']

    # Fictional constellation names (mixing Latin roots with astronomical themes)
    constellations = ['Novarum', 'Sideris', 'Caelum', 'Astralis', 'Stellara',
                      'Cosmicus', 'Galaxia', 'Nebulae', 'Quasaris', 'Pulsar',
                      'Magnetar', 'Coronae', 'Solaris', 'Lunaris', 'Orionis',
                      'Centauri', 'Draconis', 'Cygni', 'Lyrae', 'Aquilae',
                      'Pegasi', 'Andromeda', 'Cassiopeia', 'Ursae', 'Leonis']

    galactic_quadrants = ['Alpha', 'Beta', 'Gamma', 'Delta']

    star_ids = []
    print(f"🌟 Generating {num_stars} star systems...")

    for i in range(num_stars):
        # Select spectral type based on probability distribution
        rand = random.random()
        cumulative = 0
        for spec_type, prob, temp_range, lum_range, mass_range in spectral_data:
            cumulative += prob
            if rand <= cumulative:
                spectral_type = spec_type + str(random.randint(0, 9))  # Add subclass
                temperature = random.randint(temp_range[0], temp_range[1])
                luminosity = random.uniform(lum_range[0], lum_range[1])
                mass = random.uniform(mass_range[0], mass_range[1])
                break

        # Generate star name (e.g., "Alpha Novarum 7")
        name = f"{random.choice(greek_letters)} {random.choice(constellations)} {random.randint(1, 999)}"

        # Distance follows a distribution favoring nearby stars (easier to study)
        distance = random.expovariate(1/100) + 10  # Most within 100 ly, minimum 10 ly

        # Age based on spectral type (massive stars are younger)
        if spec_type in ['O', 'B']:
            age = random.uniform(0.001, 0.5)  # Very young
        elif spec_type in ['A', 'F']:
            age = random.uniform(0.5, 3)
        elif spec_type == 'G':
            age = random.uniform(2, 10)
        else:
            age = random.uniform(1, 13)  # Red dwarfs can be very old

        # Random sky coordinates
        ra = random.uniform(0, 360)
        dec = random.uniform(-90, 90)

        # Metallicity (affects planet formation)
        metallicity = random.gauss(0, 0.3)  # Solar neighborhood average

        quadrant = random.choice(galactic_quadrants)

        try:
            cursor.execute('''
                INSERT INTO STAR_SYSTEMS
                (name, spectral_type, luminosity, temperature, distance_ly,
                 age_gyr, mass_solar, ra_deg, dec_deg, galactic_quadrant, metallicity)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (name, spectral_type, round(luminosity, 4), temperature,
                  round(distance, 2), round(age, 2), round(mass, 3),
                  round(ra, 4), round(dec, 4), quadrant, round(metallicity, 3)))
            star_ids.append(cursor.lastrowid)
        except sqlite3.IntegrityError:
            # Name collision, skip
            continue

    conn.commit()
    print(f"   ✓ Created {len(star_ids)} star systems")
    return star_ids


def generate_exoplanets(conn, star_ids, avg_planets_per_star=3):
    """
    Generates realistic exoplanet data for each star system.

    Uses Kepler mission statistics for planet occurrence rates and
    size distributions.

    Args:
        conn: SQLite database connection
        star_ids: List of star system IDs to populate with planets
        avg_planets_per_star: Average number of planets per system

    Returns:
        list: List of planet_ids created
    """
    cursor = conn.cursor()
    planet_ids = []

    # Greek letters for planet naming
    planet_designators = ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k']

    print(f"🪐 Generating exoplanets for {len(star_ids)} star systems...")

    for star_id in star_ids:
        # Get star properties
        cursor.execute('''
            SELECT name, luminosity, temperature, mass_solar
            FROM STAR_SYSTEMS WHERE star_id = ?
        ''', (star_id,))
        star = cursor.fetchone()
        star_name, luminosity, temp_k, star_mass = star

        # Number of planets (Poisson distribution, some stars have none)
        num_planets = np.random.poisson(avg_planets_per_star)
        num_planets = min(num_planets, 10)  # Cap at 10 planets

        # Calculate habitable zone for this star
        hz_inner, hz_outer = calculate_habitable_zone(luminosity, temp_k)

        for j in range(num_planets):
            # Planet name
            planet_name = f"{star_name} {planet_designators[j]}"

            # Semi-major axis (orbital distance)
            # Distribution favors inner planets (detection bias)
            if random.random() < 0.3:
                # Place some planets in habitable zone intentionally
                sma = random.uniform(hz_inner * 0.8, hz_outer * 1.2)
            else:
                # Log-uniform distribution for other planets
                sma = 10 ** random.uniform(-1.5, 1.5)  # 0.03 to 30 AU

            # Orbital period from Kepler's third law: P² = a³/M
            orbital_period = 365.25 * math.sqrt((sma ** 3) / star_mass)

            # Eccentricity (most planets have low eccentricity)
            eccentricity = abs(random.gauss(0.05, 0.15))
            eccentricity = min(eccentricity, 0.9)  # Cap at 0.9

            # Planet size distribution (based on Kepler statistics)
            size_rand = random.random()
            if size_rand < 0.40:
                radius = random.uniform(0.5, 1.5)      # Earth-sized
            elif size_rand < 0.70:
                radius = random.uniform(1.5, 2.5)      # Super-Earths
            elif size_rand < 0.85:
                radius = random.uniform(2.5, 4)        # Mini-Neptunes
            elif size_rand < 0.95:
                radius = random.uniform(4, 11)         # Neptune to Jupiter
            else:
                radius = random.uniform(11, 25)        # Super-Jupiters

            # Mass from radius (using empirical mass-radius relations)
            if radius < 1.5:
                mass = radius ** 3.5  # Rocky composition
            elif radius < 4:
                mass = radius ** 2.5  # Mix of rock and volatiles
            else:
                mass = 30 * (radius / 4) ** 2  # Gas giants

            # Add some scatter
            mass *= random.uniform(0.7, 1.5)

            # Calculate equilibrium temperature
            star_radius = star_mass ** 0.8  # Approximate for main sequence
            eq_temp = calculate_equilibrium_temperature(temp_k, star_radius, sma)

            # Surface gravity (relative to Earth)
            surface_gravity = mass / (radius ** 2)

            # Density (relative to Earth: 5.5 g/cm³)
            density = (mass / (radius ** 3)) * 5.5

            # Classify planet type
            planet_type = classify_planet_type(mass, radius, eq_temp)

            # Discovery date (random within last 30 years)
            days_ago = random.randint(0, 30 * 365)
            discovery_date = datetime.now() - timedelta(days=days_ago)

            try:
                cursor. execute('''
                    INSERT INTO EXOPLANETS
                    (star_id, name, mass_earth, radius_earth, orbital_period_days,
                     semi_major_axis_au, eccentricity, equilibrium_temp_k,
                     surface_gravity_earth, density_gcc, planet_type,
                     discovery_date, confirmed)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (star_id, planet_name, round(mass, 3), round(radius, 3),
                      round(orbital_period, 2), round(sma, 4), round(eccentricity, 4),
                      round(eq_temp, 1), round(surface_gravity, 3), round(density, 3),
                      planet_type, discovery_date.strftime('%Y-%m-%d'),
                      random.random() > 0.05))  # 95% confirmed
                planet_ids. append(cursor.lastrowid)
            except sqlite3.IntegrityError:
                continue

    conn.commit()
    print(f"   ✓ Created {len(planet_ids)} exoplanets")
    return planet_ids


def generate_discovery_missions(conn):
    """
    Creates records for real and fictional space missions and observatories.

    Includes actual missions like Kepler, TESS, and James Webb, plus
    fictional future missions for variety.

    Args:
        conn: SQLite database connection

    Returns:
        list: List of mission_ids created
    """
    cursor = conn.cursor()

    # Mix of real and fictional missions
    missions = [
        # Real missions
        ('Kepler Space Telescope', 'NASA', 'Space Telescope', 'Transit',
         '2009-03-07', '2018-10-30', 'Retired', 10),
        ('TESS', 'NASA', 'Space Telescope', 'Transit',
         '2018-04-18', None, 'Active', 9),
        ('James Webb Space Telescope', 'NASA/ESA/CSA', 'Space Telescope',
         'Direct Imaging', '2021-12-25', None, 'Active', 10),
        ('HARPS', 'ESO', 'Ground Observatory', 'Radial Velocity',
         '2003-02-01', None, 'Active', 9),
        ('ESPRESSO', 'ESO', 'Ground Observatory', 'Radial Velocity',
         '2018-09-01', None, 'Active', 10),
        ('Gaia', 'ESA', 'Space Telescope', 'Astrometry',
         '2013-12-19', None, 'Active', 8),

        # Fictional future missions
        ('ARIEL', 'ESA', 'Space Telescope', 'Transit Spectroscopy',
         '2029-01-01', None, 'Under Development', 9),
        ('PLATO', 'ESA', 'Space Telescope', 'Transit',
         '2026-12-01', None, 'Under Development', 10),
        ('HabEx', 'NASA', 'Space Telescope', 'Direct Imaging',
         '2035-01-01', None, 'Proposed', 10),
        ('LUVOIR', 'NASA', 'Space Telescope', 'Multi-method',
         '2039-01-01', None, 'Proposed', 10),
        ('Terran Exoplanet Explorer', 'UNOOSA', 'Space Telescope', 'Interferometry',
         '2045-01-01', None, 'Conceptual', 10),
        ('Stellar Lighthouse Array', 'International Consortium', 'Space Array',
         'Multi-method', '2050-01-01', None, 'Conceptual', 10),
    ]

    print("🚀 Generating discovery missions...")
    mission_ids = []

    for mission in missions:
        name, org, m_type, method, launch, end, status, sens = mission
        cursor.execute('''
            INSERT INTO DISCOVERY_MISSIONS
            (name, organization, mission_type, detection_method, launch_date,
             end_date, status, sensitivity_rating)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (name, org, m_type, method, launch, end, status, sens))
        mission_ids.append(cursor.lastrowid)

    conn.commit()
    print(f"   ✓ Created {len(mission_ids)} discovery missions")
    return mission_ids


def generate_planet_discoveries(conn, planet_ids, mission_ids):
    """
    Links planets to their discovery missions with detection metadata.

    Args:
        conn: SQLite database connection
        planet_ids: List of planet IDs to link
        mission_ids: List of mission IDs to use

    Returns:
        int: Number of discovery records created
    """
    cursor = conn.cursor()

    discovery_teams = [
        'MIT Exoplanet Team', 'Caltech Planet Hunters', 'Geneva Observatory',
        'Cambridge Biosignature Group', 'Tokyo Astrobiology Institute',
        'ESO Spectroscopy Division', 'NASA Ames Research Center',
        'Max Planck Planetology', 'University of Hawai\'i IFA',
        'SETI Institute Research', 'European Space Astronomy Centre'
    ]

    roles = ['Primary Discoverer', 'Confirmation', 'Follow-up', 'Characterization']

    print("🔭 Generating discovery records...")
    count = 0

    for planet_id in planet_ids:
        # Get planet discovery date
        cursor.execute('SELECT discovery_date FROM EXOPLANETS WHERE planet_id = ?',
                       (planet_id,))
        disc_date = cursor.fetchone()[0]

        # Each planet has 1-3 discovery/confirmation records
        num_records = random.randint(1, 3)

        for i in range(num_records):
            mission_id = random.choice(mission_ids)
            role = roles[0] if i == 0 else random.choice(roles[1:])
            snr = random.uniform(3, 50)  # Signal-to-noise ratio
            transit_events = random.randint(3, 50) if role == 'Primary Discoverer' else None
            team = random.choice(discovery_teams)
            pub_ref = f"arXiv:{random.randint(1900, 2500)}. {random.randint(10000, 99999)}" if random.random() > 0.2 else None

            cursor.execute('''
                INSERT INTO PLANET_DISCOVERIES
                (planet_id, mission_id, detection_date, discovery_role,
                 snr_value, transit_events, discovery_team, publication_ref)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (planet_id, mission_id, disc_date, role, round(snr, 2),
                  transit_events, team, pub_ref))
            count += 1

    conn.commit()
    print(f"   ✓ Created {count} discovery records")
    return count


def generate_atmospheric_analysis(conn, planet_ids):
    """
    Generates atmospheric composition data for a subset of planets.

    Only planets that have been characterized (typically larger, closer planets)
    have atmospheric data available.

    Args:
        conn: SQLite database connection
        planet_ids: List of planet IDs

    Returns:
        int: Number of atmospheric analyses created
    """
    cursor = conn.cursor()

    print("🌫️ Generating atmospheric analyses...")
    count = 0

    # Only about 20% of planets have atmospheric characterization
    analyzed_planets = random.sample(planet_ids, k=int(len(planet_ids) * 0.2))

    for planet_id in analyzed_planets:
        # Get planet type
        cursor.execute('SELECT planet_type, mass_earth, equilibrium_temp_k FROM EXOPLANETS WHERE planet_id = ?',
                       (planet_id,))
        planet_type, mass, temp = cursor.fetchone()

        # Composition depends on planet type
        if 'Terrestrial' in planet_type or 'Super-Earth' in planet_type:
            # Rocky planet atmosphere
            if temp > 350:  # Hot
                h = random.uniform(0, 10)
                he = random.uniform(0, 5)
                n2 = random.uniform(0, 20)
                o2 = random.uniform(0, 5)
                co2 = random.uniform(50, 95)
                ch4 = random.uniform(0, 2)
                h2o = random.uniform(0, 10)
            elif 200 < temp < 350:  # Temperate (potentially habitable)
                h = random.uniform(0, 2)
                he = random.uniform(0, 1)
                n2 = random.uniform(60, 85)
                o2 = random.uniform(0, 25)  # Could have oxygen!
                co2 = random.uniform(0.01, 5)
                ch4 = random.uniform(0, 2)
                h2o = random.uniform(0.1, 5)
            else:  # Cold
                h = random.uniform(0, 5)
                he = random.uniform(0, 2)
                n2 = random.uniform(80, 99)
                o2 = random.uniform(0, 3)
                co2 = random.uniform(0, 1)
                ch4 = random.uniform(0, 5)
                h2o = random.uniform(0, 1)
        else:
            # Gas giant atmosphere
            h = random.uniform(70, 95)
            he = random.uniform(5, 25)
            n2 = random.uniform(0, 2)
            o2 = random.uniform(0, 0.5)
            co2 = random.uniform(0, 1)
            ch4 = random.uniform(0.1, 3)
            h2o = random.uniform(0, 2)

        # Normalize to ~100%
        total = h + he + n2 + o2 + co2 + ch4 + h2o
        factor = 97 / total  # Leave some for others
        h *= factor
        he *= factor
        n2 *= factor
        o2 *= factor
        co2 *= factor
        ch4 *= factor
        h2o *= factor

        nh3 = random.uniform(0, 1)
        ar = random.uniform(0, 1.5)
        other = 100 - (h + he + n2 + o2 + co2 + ch4 + h2o + nh3 + ar)

        # Surface pressure
        if 'Terrestrial' in planet_type or 'Super-Earth' in planet_type:
            pressure = random.uniform(0.1, 100) * (mass ** 0.5)
        else:
            pressure = None  # No defined surface

        # Biosignature detection (very rare, requires specific conditions)
        biosignature = (o2 > 10 and ch4 > 0.5 and 200 < temp < 350 and
                        random.random() < 0.1)

        stability = random.choice(['Stable', 'Moderate', 'Unstable', 'Unknown'])
        greenhouse = min(10, int((co2 + ch4 + h2o) / 5) + random.randint(1, 3))

        analysis_date = datetime.now() - timedelta(days=random.randint(0, 1000))

        cursor.execute('''
            INSERT INTO ATMOSPHERIC_ANALYSIS
            (planet_id, analysis_date, hydrogen_pct, helium_pct, nitrogen_pct,
             oxygen_pct, carbon_dioxide_pct, methane_pct, water_vapor_pct,
             ammonia_pct, argon_pct, other_pct, surface_pressure_atm,
             biosignature_detected, atmosphere_stability, greenhouse_rating)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (planet_id, analysis_date.strftime('%Y-%m-%d'),
              round(h, 2), round(he, 2), round(n2, 2), round(o2, 2),
              round(co2, 2), round(ch4, 2), round(h2o, 2), round(nh3, 2),
              round(ar, 2), round(max(0, other), 2),
              round(pressure, 2) if pressure else None,
              1 if biosignature else 0, stability, greenhouse))
        count += 1

    conn.commit()
    print(f"   ✓ Created {count} atmospheric analyses")
    return count


def generate_habitability_scores(conn, planet_ids):
    """
    Calculates and stores habitability scores for all planets.

    Uses the ESI formula and other metrics to assess habitability potential.

    Args:
        conn: SQLite database connection
        planet_ids: List of planet IDs

    Returns:
        int: Number of habitability scores created
    """
    cursor = conn.cursor()

    print("🌍 Calculating habitability scores...")
    count = 0

    for planet_id in planet_ids:
        # Get planet and star data
        cursor.execute('''
            SELECT e.mass_earth, e.radius_earth, e.equilibrium_temp_k,
                   e.semi_major_axis_au, e.surface_gravity_earth, e.planet_type,
                   s.luminosity, s.temperature
            FROM EXOPLANETS e
            JOIN STAR_SYSTEMS s ON e.star_id = s.star_id
            WHERE e.planet_id = ?
        ''', (planet_id,))

        planet_data = cursor.fetchone()
        mass, radius, temp, sma, gravity, p_type, lum, star_temp = planet_data

        # Calculate ESI
        escape_vel = math.sqrt(mass / radius) if radius > 0 else 1
        esi = calculate_esi(radius, mass, temp, escape_vel)

        # Determine HZ status
        hz_inner, hz_outer = calculate_habitable_zone(lum, star_temp)
        if hz_inner <= sma <= hz_outer:
            hz_status = "In HZ"
        elif hz_inner * 0.8 <= sma <= hz_outer * 1.2:
            hz_status = "Near HZ Edge"
        elif sma < hz_inner:
            hz_status = "Too Hot"
        else:
            hz_status = "Too Cold"

        # Water probability
        if hz_status == "In HZ" and 200 < temp < 350:
            water_prob = min(95, 50 + esi * 50)
        elif "Near" in hz_status:
            water_prob = min(60, 20 + esi * 40)
        else:
            water_prob = max(0, esi * 20 - 10)

        # Check for atmospheric data
        cursor.execute('SELECT oxygen_pct, greenhouse_rating FROM ATMOSPHERIC_ANALYSIS WHERE planet_id = ?',
                       (planet_id,))
        atm_data = cursor.fetchone()

        if atm_data:
            o2, greenhouse = atm_data
            atm_rating = min(10, int(3 + o2/5 + (10 - abs(greenhouse - 5))/2))
        else:
            atm_rating = 5  # Unknown, assume average

        # Magnetic field probability (based on mass and rotation)
        mag_prob = min(90, 10 + mass * 20) if mass < 3 else max(10, 50 - mass)

        # Tidal lock probability (closer planets more likely locked)
        tidal_prob = max(0, min(100, 80 - sma * 50))

        # Classify habitability
        hab_class = classify_habitability(esi, hz_status, water_prob, atm_rating)

        # Study priority (1-10, lower is higher priority)
        if "Class I" in hab_class:
            priority = random.randint(1, 2)
        elif "Class II" in hab_class:
            priority = random.randint(2, 4)
        elif "Class III" in hab_class:
            priority = random.randint(4, 6)
        else:
            priority = random.randint(6, 10)

        cursor.execute('''
            INSERT INTO HABITABILITY_SCORES
            (planet_id, esi_score, hz_status, water_probability, atmosphere_rating,
             magnetic_field_probability, tidal_lock_probability,
             habitability_class, study_priority)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (planet_id, round(esi, 4), hz_status, round(water_prob, 1),
              atm_rating, round(mag_prob, 1), round(tidal_prob, 1),
              hab_class, priority))
        count += 1

    conn.commit()
    print(f"   ✓ Created {count} habitability scores")
    return count


def populate_database(conn):
    """
    Master function to populate the entire database with mock data.

    Args:
        conn: SQLite database connection

    Returns:
        dict: Statistics about generated data
    """
    print("\n" + "=" * 60)
    print("📊 BATCH DATA GENERATION STARTING")
    print("=" * 60)

    # Generate data in dependency order
    star_ids = generate_star_systems(conn, num_stars=150)
    planet_ids = generate_exoplanets(conn, star_ids, avg_planets_per_star=3)
    mission_ids = generate_discovery_missions(conn)
    discovery_count = generate_planet_discoveries(conn, planet_ids, mission_ids)
    atm_count = generate_atmospheric_analysis(conn, planet_ids)
    hab_count = generate_habitability_scores(conn, planet_ids)

    # Update mission discovery counts
    cursor = conn.cursor()
    cursor.execute('''
        UPDATE DISCOVERY_MISSIONS
        SET total_discoveries = (
            SELECT COUNT(DISTINCT planet_id)
            FROM PLANET_DISCOVERIES pd
            WHERE pd.mission_id = DISCOVERY_MISSIONS.mission_id
            AND pd.discovery_role = 'Primary Discoverer'
        )
    ''')
    conn.commit()

    stats = {
        'stars': len(star_ids),
        'planets': len(planet_ids),
        'missions': len(mission_ids),
        'discoveries': discovery_count,
        'atmospheres': atm_count,
        'habitability': hab_count
    }

    print("\n" + "=" * 60)
    print("✅ DATABASE POPULATION COMPLETE")
    print("=" * 60)
    print(f"   Stars:                {stats['stars']:,}")
    print(f"   Planets:              {stats['planets']:,}")
    print(f"   Missions:             {stats['missions']:,}")
    print(f"   Discovery Records:    {stats['discoveries']:,}")
    print(f"   Atmospheric Analyses: {stats['atmospheres']:,}")
    print(f"   Habitability Scores:  {stats['habitability']:,}")
    print("=" * 60 + "\n")

    return stats

# %%
# =============================================================================
# SECTION 5: ADVANCED SQL QUERY FUNCTIONS
# =============================================================================
# Complex queries demonstrating aggregations, joins, window functions, and subqueries

def get_habitability_summary(conn):
    """
    Returns a summary of planets grouped by habitability class.

    Uses:
        - Aggregation functions (COUNT, AVG, MIN, MAX)
        - JOIN between multiple tables
        - GROUP BY with multiple metrics

    Args:
        conn: SQLite database connection

    Returns:
        pandas.DataFrame: Summary statistics by habitability class
    """
    query = '''
        SELECT
            h.habitability_class,
            COUNT(*) as planet_count,
            ROUND(AVG(h.esi_score), 4) as avg_esi,
            ROUND(MIN(h.esi_score), 4) as min_esi,
            ROUND(MAX(h.esi_score), 4) as max_esi,
            ROUND(AVG(h.water_probability), 1) as avg_water_prob,
            ROUND(AVG(e.equilibrium_temp_k), 0) as avg_temp_k,
            ROUND(AVG(e.radius_earth), 2) as avg_radius,
            ROUND(AVG(e.mass_earth), 2) as avg_mass
        FROM HABITABILITY_SCORES h
        JOIN EXOPLANETS e ON h.planet_id = e.planet_id
        GROUP BY h.habitability_class
        ORDER BY
            CASE
                WHEN h.habitability_class LIKE '%Class I%' THEN 1
                WHEN h.habitability_class LIKE '%Class II%' THEN 2
                WHEN h.habitability_class LIKE '%Class III%' THEN 3
                WHEN h.habitability_class LIKE '%Class IV%' THEN 4
                ELSE 5
            END
    '''
    return pd.read_sql_query(query, conn)


def get_top_candidates(conn, limit=20):
    """
    Returns the top habitability candidates with full details.

    Uses:
        - Multi-table JOIN (4 tables)
        - Subquery for ranking
        - Complex ORDER BY

    Args:
        conn: SQLite database connection
        limit: Maximum number of results

    Returns:
        pandas.DataFrame: Top habitable planet candidates
    """
    query = f'''
        SELECT
            e.name as planet_name,
            s.name as host_star,
            s.spectral_type,
            s.distance_ly,
            e.planet_type,
            ROUND(e.radius_earth, 2) as radius_earth,
            ROUND(e.mass_earth, 2) as mass_earth,
            ROUND(e.equilibrium_temp_k, 0) as temp_k,
            h.hz_status,
            ROUND(h.esi_score, 4) as esi_score,
            ROUND(h.water_probability, 1) as water_prob,
            h.habitability_class,
            h.study_priority,
            CASE WHEN a.biosignature_detected = 1 THEN 'Yes' ELSE 'No' END as biosignature
        FROM EXOPLANETS e
        JOIN STAR_SYSTEMS s ON e.star_id = s.star_id
        JOIN HABITABILITY_SCORES h ON e.planet_id = h.planet_id
        LEFT JOIN ATMOSPHERIC_ANALYSIS a ON e.planet_id = a.planet_id
        WHERE h.habitability_class IN ('Class I: Prime Candidate', 'Class II: High Potential')
        ORDER BY h.esi_score DESC, h.water_probability DESC
        LIMIT {limit}
    '''
    return pd.read_sql_query(query, conn)


def get_mission_performance_ranking(conn):
    """
    Ranks discovery missions by their effectiveness using window functions.

    Uses:
        - Window function (RANK, DENSE_RANK)
        - Aggregation with GROUP BY
        - CASE expressions
        - Multiple JOINs

    Args:
        conn: SQLite database connection

    Returns:
        pandas.DataFrame: Mission performance rankings
    """
    query = '''
        WITH mission_stats AS (
            SELECT
                m.mission_id,
                m.name as mission_name,
                m.organization,
                m.detection_method,
                m.status,
                COUNT(DISTINCT pd.planet_id) as total_planets,
                SUM(CASE WHEN pd.discovery_role = 'Primary Discoverer' THEN 1 ELSE 0 END) as primary_discoveries,
                SUM(CASE WHEN h.habitability_class LIKE '%Class I%' OR h.habitability_class LIKE '%Class II%' THEN 1 ELSE 0 END) as habitable_candidates,
                AVG(h.esi_score) as avg_esi
            FROM DISCOVERY_MISSIONS m
            LEFT JOIN PLANET_DISCOVERIES pd ON m.mission_id = pd.mission_id
            LEFT JOIN HABITABILITY_SCORES h ON pd.planet_id = h.planet_id
            GROUP BY m.mission_id, m.name, m.organization, m.detection_method, m.status
        )
        SELECT
            mission_name,
            organization,
            detection_method,
            status,
            total_planets,
            primary_discoveries,
            habitable_candidates,
            ROUND(avg_esi, 4) as avg_esi_score,
            RANK() OVER (ORDER BY habitable_candidates DESC, primary_discoveries DESC) as overall_rank,
            DENSE_RANK() OVER (PARTITION BY status ORDER BY total_planets DESC) as rank_in_status
        FROM mission_stats
        ORDER BY overall_rank
    '''
    return pd.read_sql_query(query, conn)


def get_stellar_type_analysis(conn):
    """
    Analyzes planet occurrence and habitability by stellar type.

    Uses:
        - Complex aggregations
        - Subqueries
        - CASE expressions for conditional counting

    Args:
        conn: SQLite database connection

    Returns:
        pandas.DataFrame: Analysis by stellar spectral type
    """
    query = '''
        SELECT
            SUBSTR(s.spectral_type, 1, 1) as spectral_class,
            COUNT(DISTINCT s.star_id) as num_stars,
            COUNT(e.planet_id) as total_planets,
            ROUND(CAST(COUNT(e.planet_id) AS REAL) / COUNT(DISTINCT s.star_id), 2) as planets_per_star,
            SUM(CASE WHEN h.hz_status = 'In HZ' THEN 1 ELSE 0 END) as planets_in_hz,
            SUM(CASE WHEN h.habitability_class LIKE '%Class I%' THEN 1 ELSE 0 END) as class_i_count,
            SUM(CASE WHEN h.habitability_class LIKE '%Class II%' THEN 1 ELSE 0 END) as class_ii_count,
            ROUND(AVG(h.esi_score), 4) as avg_esi,
            ROUND(AVG(s.luminosity), 4) as avg_luminosity,
            ROUND(AVG(s.temperature), 0) as avg_temperature
        FROM STAR_SYSTEMS s
        LEFT JOIN EXOPLANETS e ON s.star_id = e.star_id
        LEFT JOIN HABITABILITY_SCORES h ON e.planet_id = h.planet_id
        GROUP BY SUBSTR(s.spectral_type, 1, 1)
        ORDER BY
            CASE SUBSTR(s.spectral_type, 1, 1)
                WHEN 'O' THEN 1
                WHEN 'B' THEN 2
                WHEN 'A' THEN 3
                WHEN 'F' THEN 4
                WHEN 'G' THEN 5
                WHEN 'K' THEN 6
                WHEN 'M' THEN 7
            END
    '''
    return pd.read_sql_query(query, conn)


def get_discovery_timeline(conn):
    """
    Returns discovery counts aggregated by year for trend analysis.

    Uses:
        - Date functions (strftime)
        - Cumulative sum via window function
        - GROUP BY with date extraction

    Args:
        conn: SQLite database connection

    Returns:
        pandas.DataFrame: Discovery timeline data
    """
    query = '''
        SELECT
            strftime('%Y', e.discovery_date) as discovery_year,
            COUNT(*) as discoveries,
            SUM(CASE WHEN e.confirmed = 1 THEN 1 ELSE 0 END) as confirmed,
            SUM(CASE WHEN h.hz_status = 'In HZ' THEN 1 ELSE 0 END) as in_habitable_zone,
            SUM(CASE WHEN h.habitability_class LIKE '%Class I%' OR
                          h.habitability_class LIKE '%Class II%' THEN 1 ELSE 0 END) as high_priority,
            ROUND(AVG(h.esi_score), 4) as avg_esi,
            SUM(COUNT(*)) OVER (ORDER BY strftime('%Y', e.discovery_date)) as cumulative_total
        FROM EXOPLANETS e
        LEFT JOIN HABITABILITY_SCORES h ON e.planet_id = h.planet_id
        GROUP BY strftime('%Y', e.discovery_date)
        ORDER BY discovery_year
    '''
    return pd.read_sql_query(query, conn)


def get_atmospheric_biosignatures(conn):
    """
    Identifies planets with potential biosignature detections.

    Uses:
        - Complex WHERE conditions
        - Multiple table JOINs
        - Calculated fields

    Args:
        conn: SQLite database connection

    Returns:
        pandas.DataFrame: Planets with biosignature potential
    """
    query = '''
        SELECT
            e.name as planet_name,
            s.name as host_star,
            s.distance_ly,
            e.planet_type,
            ROUND(e.equilibrium_temp_k, 0) as temp_k,
            ROUND(a.oxygen_pct, 2) as oxygen_pct,
            ROUND(a.methane_pct, 2) as methane_pct,
            ROUND(a.water_vapor_pct, 2) as water_vapor_pct,
            ROUND(a.carbon_dioxide_pct, 2) as co2_pct,
            a.biosignature_detected,
            a.atmosphere_stability,
            h.hz_status,
            ROUND(h.esi_score, 4) as esi_score,
            h.habitability_class,
            -- Calculate a biosignature potential score
            ROUND(
                (a.oxygen_pct * 2 + a.methane_pct * 5 + a.water_vapor_pct * 1.5) *
                CASE WHEN h.hz_status = 'In HZ' THEN 1.5 ELSE 0.5 END,
                2
            ) as biosig_potential_score
        FROM ATMOSPHERIC_ANALYSIS a
        JOIN EXOPLANETS e ON a.planet_id = e.planet_id
        JOIN STAR_SYSTEMS s ON e.star_id = s.star_id
        JOIN HABITABILITY_SCORES h ON e.planet_id = h.planet_id
        WHERE a.oxygen_pct > 1 OR a.methane_pct > 0.5 OR a.biosignature_detected = 1
        ORDER BY biosig_potential_score DESC
        LIMIT 30
    '''
    return pd.read_sql_query(query, conn)


def get_planet_type_distribution(conn):
    """
    Returns distribution of planet types with habitability metrics.

    Args:
        conn: SQLite database connection

    Returns:
        pandas.DataFrame: Planet type distribution
    """
    query = '''
        SELECT
            e.planet_type,
            COUNT(*) as count,
            ROUND(AVG(e.radius_earth), 2) as avg_radius,
            ROUND(AVG(e.mass_earth), 2) as avg_mass,
            ROUND(AVG(e.equilibrium_temp_k), 0) as avg_temp,
            ROUND(AVG(h.esi_score), 4) as avg_esi,
            SUM(CASE WHEN h.hz_status = 'In HZ' THEN 1 ELSE 0 END) as in_hz_count,
            ROUND(100.0 * SUM(CASE WHEN h.hz_status = 'In HZ' THEN 1 ELSE 0 END) / COUNT(*), 1) as hz_percentage
        FROM EXOPLANETS e
        JOIN HABITABILITY_SCORES h ON e.planet_id = h.planet_id
        GROUP BY e.planet_type
        ORDER BY count DESC
    '''
    return pd.read_sql_query(query, conn)


def search_planets(conn, min_esi=None, max_distance=None, planet_type=None,
                   hz_only=False, min_water_prob=None, spectral_type=None):
    """
    Flexible search function with multiple filter criteria.

    Demonstrates dynamic SQL query construction.

    Args:
        conn: SQLite database connection
        min_esi: Minimum ESI score filter
        max_distance: Maximum distance in light-years
        planet_type: Filter by planet type (partial match)
        hz_only: If True, only return planets in habitable zone
        min_water_prob: Minimum water probability percentage
        spectral_type: Filter by host star spectral type (first letter)

    Returns:
        pandas.DataFrame: Matching planets
    """
    base_query = '''
        SELECT
            e.name as planet_name,
            s.name as host_star,
            s.spectral_type,
            ROUND(s.distance_ly, 1) as distance_ly,
            e.planet_type,
            ROUND(e.radius_earth, 2) as radius,
            ROUND(e.mass_earth, 2) as mass,
            ROUND(e.equilibrium_temp_k, 0) as temp_k,
            h.hz_status,
            ROUND(h.esi_score, 4) as esi,
            ROUND(h.water_probability, 1) as water_prob,
            h.habitability_class
        FROM EXOPLANETS e
        JOIN STAR_SYSTEMS s ON e.star_id = s.star_id
        JOIN HABITABILITY_SCORES h ON e.planet_id = h.planet_id
        WHERE 1=1
    '''

    params = []

    if min_esi is not None:
        base_query += " AND h.esi_score >= ?"
        params.append(min_esi)

    if max_distance is not None:
        base_query += " AND s.distance_ly <= ?"
        params.append(max_distance)

    if planet_type is not None:
        base_query += " AND e.planet_type LIKE ?"
        params.append(f"%{planet_type}%")

    if hz_only:
        base_query += " AND h.hz_status = 'In HZ'"

    if min_water_prob is not None:
        base_query += " AND h.water_probability >= ?"
        params.append(min_water_prob)

    if spectral_type is not None:
        base_query += " AND s.spectral_type LIKE ?"
        params.append(f"{spectral_type}%")

    base_query += " ORDER BY h.esi_score DESC LIMIT 100"

    return pd.read_sql_query(base_query, conn, params=params)


print("✅ Advanced SQL query functions defined!")

# %%
# =============================================================================
# SECTION 6: DATA VISUALIZATION FUNCTIONS
# =============================================================================
# Create compelling visualizations for exoplanet data

def plot_habitability_distribution(conn):
    """
    Creates a pie chart showing distribution of habitability classes.
    """
    df = get_habitability_summary(conn)

    # Custom colors for habitability classes
    colors = ['#00ff88', '#88ff00', '#ffff00', '#ff8800', '#ff0044']

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # Pie chart
    wedges, texts, autotexts = ax1.pie(
        df['planet_count'],
        labels=df['habitability_class'].apply(lambda x: x.split(':')[0]),
        autopct='%1.1f%%',
        colors=colors,
        explode=[0.1, 0.05, 0, 0, 0],
        shadow=True,
        startangle=90
    )
    ax1.set_title('Distribution of Habitability Classes', fontsize=14, fontweight='bold')

    # Bar chart for average ESI by class
    bars = ax2.barh(df['habitability_class'], df['avg_esi'], color=colors)
    ax2.set_xlabel('Average ESI Score', fontsize=12)
    ax2.set_title('Average Earth Similarity Index by Class', fontsize=14, fontweight='bold')
    ax2.set_xlim(0, 1)

    # Add value labels on bars
    for bar, val in zip(bars, df['avg_esi']):
        ax2.text(val + 0.02, bar.get_y() + bar.get_height()/2,
                f'{val:.3f}', va='center', fontsize=10)

    plt.tight_layout()
    plt.show()

    return df


def plot_stellar_habitability_heatmap(conn):
    """
    Creates a heatmap showing habitability potential by stellar type.
    """
    df = get_stellar_type_analysis(conn)

    fig, ax = plt.subplots(figsize=(12, 8))

    # Prepare data for heatmap
    metrics = ['planets_per_star', 'planets_in_hz', 'class_i_count',
               'class_ii_count', 'avg_esi']
    metric_labels = ['Planets/Star', 'In HZ', 'Class I', 'Class II', 'Avg ESI']

    data_matrix = df[metrics].values.T

    # Normalize each row to 0-1 for better visualization
    normalized = np.zeros_like(data_matrix, dtype=float)
    for i in range(len(metrics)):
        row = data_matrix[i].astype(float)
        if row.max() > row.min():
            normalized[i] = (row - row.min()) / (row.max() - row.min())
        else:
            normalized[i] = 0.5

    # Create custom colormap (space-themed)
    colors_cmap = ['#0a0a2e', '#1a1a5e', '#3366cc', '#66ccff', '#00ff88']
    cmap = LinearSegmentedColormap.from_list('space', colors_cmap)

    im = ax.imshow(normalized, cmap=cmap, aspect='auto')

    # Labels
    ax.set_xticks(range(len(df)))
    ax.set_xticklabels(df['spectral_class'], fontsize=12)
    ax.set_yticks(range(len(metric_labels)))
    ax.set_yticklabels(metric_labels, fontsize=11)

    # Add text annotations with actual values
    for i in range(len(metric_labels)):
        for j in range(len(df)):
            val = data_matrix[i][j]
            if metrics[i] == 'avg_esi':
                text = f'{val:.3f}'
            elif metrics[i] == 'planets_per_star':
                text = f'{val:.1f}'
            else:
                text = f'{int(val)}'
            ax.text(j, i, text, ha='center', va='center',
                   color='white' if normalized[i][j] < 0.5 else 'black',
                   fontsize=10, fontweight='bold')

    ax.set_title('Habitability Metrics by Stellar Spectral Type\n(O-B-A-F-G-K-M: Hot → Cool)',
                fontsize=14, fontweight='bold')
    ax.set_xlabel('Spectral Type', fontsize=12)

    # Colorbar
    cbar = plt.colorbar(im, ax=ax, shrink=0.8)
    cbar.set_label('Normalized Value', fontsize=11)

    plt.tight_layout()
    plt.show()

    return df


def plot_discovery_timeline(conn):
    """
    Creates a timeline visualization of exoplanet discoveries.
    """
    df = get_discovery_timeline(conn)

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

    years = df['discovery_year'].astype(str)

    # Stacked bar chart for discoveries
    width = 0.8
    ax1.bar(years, df['confirmed'], width, label='Confirmed', color='#00ff88', alpha=0.8)
    ax1.bar(years, df['discoveries'] - df['confirmed'], width,
           bottom=df['confirmed'], label='Unconfirmed', color='#ff8844', alpha=0.8)

    # Overlay line for high priority discoveries
    ax1_twin = ax1.twinx()
    ax1_twin.plot(years, df['high_priority'], 'o-', color='#ff00ff',
                 linewidth=2, markersize=8, label='High Priority Candidates')
    ax1_twin.set_ylabel('High Priority Count', fontsize=11, color='#ff00ff')
    ax1_twin.tick_params(axis='y', labelcolor='#ff00ff')

    ax1.set_ylabel('Number of Discoveries', fontsize=11)
    ax1.set_title('Exoplanet Discoveries by Year', fontsize=14, fontweight='bold')
    ax1.legend(loc='upper left')
    ax1_twin.legend(loc='upper right')

    # Cumulative discoveries with ESI trend
    ax2.fill_between(years, df['cumulative_total'], alpha=0.3, color='#00aaff')
    ax2.plot(years, df['cumulative_total'], 'o-', color='#00aaff',
            linewidth=2, markersize=6, label='Cumulative Total')

    ax2_twin = ax2.twinx()
    ax2_twin.plot(years, df['avg_esi'], 's--', color='#ffff00',
                 linewidth=2, markersize=8, label='Average ESI')
    ax2_twin.set_ylabel('Average ESI Score', fontsize=11, color='#ffff00')
    ax2_twin.tick_params(axis='y', labelcolor='#ffff00')
    ax2_twin.set_ylim(0, 1)

    ax2.set_xlabel('Discovery Year', fontsize=12)
    ax2.set_ylabel('Cumulative Discoveries', fontsize=11)
    ax2.set_title('Cumulative Discovery Growth & ESI Trend', fontsize=14, fontweight='bold')
    ax2.legend(loc='upper left')
    ax2_twin.legend(loc='center right')

    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    return df


def plot_planet_scatter(conn):
    """
    Creates a scatter plot of planets by mass, radius, and temperature.
    Similar to diagrams used by astronomers to classify exoplanets.
    """
    query = '''
        SELECT
            e.name,
            e.mass_earth,
            e.radius_earth,
            e.equilibrium_temp_k,
            e.planet_type,
            h.esi_score,
            h.hz_status,
            h.habitability_class
        FROM EXOPLANETS e
        JOIN HABITABILITY_SCORES h ON e.planet_id = h.planet_id
    '''
    df = pd.read_sql_query(query, conn)

    fig, ax = plt.subplots(figsize=(14, 10))

    # Color by ESI score
    scatter = ax.scatter(
        df['mass_earth'],
        df['radius_earth'],
        c=df['esi_score'],
        s=df['equilibrium_temp_k'] / 10,  # Size by temperature
        cmap='plasma',
        alpha=0.7,
        edgecolors='white',
        linewidth=0.5
    )

    # Mark habitable zone planets
    hz_planets = df[df['hz_status'] == 'In HZ']
    ax.scatter(
        hz_planets['mass_earth'],
        hz_planets['radius_earth'],
        facecolors='none',
        edgecolors='#00ff00',
        s=200,
        linewidth=2,
        marker='o',
        label='In Habitable Zone'
    )

    # Reference lines for Earth, Neptune, Jupiter
    ax.axhline(y=1, color='#00aaff', linestyle='--', alpha=0.5, label='Earth radius')
    ax.axvline(x=1, color='#00aaff', linestyle='--', alpha=0.5, label='Earth mass')
    ax.axhline(y=3.88, color='#00ff88', linestyle=':', alpha=0.5, label='Neptune radius')
    ax.axhline(y=11.2, color='#ff8800', linestyle=':', alpha=0.5, label='Jupiter radius')

    # Log scale for better distribution visibility
    ax.set_xscale('log')
    ax.set_yscale('log')

    ax.set_xlabel('Mass (Earth masses)', fontsize=12)
    ax.set_ylabel('Radius (Earth radii)', fontsize=12)
    ax.set_title('Exoplanet Mass-Radius Diagram\n(Color: ESI Score, Size: Temperature)',
                fontsize=14, fontweight='bold')

    # Colorbar
    cbar = plt.colorbar(scatter, ax=ax, shrink=0.8)
    cbar.set_label('Earth Similarity Index (ESI)', fontsize=11)

    ax.legend(loc='upper left', fontsize=10)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    return df


def plot_atmospheric_composition(conn, planet_name=None):
    """
    Creates radar charts showing atmospheric composition of interesting planets.
    """
    if planet_name:
        query = f'''
            SELECT
                e.name,
                a.hydrogen_pct, a.helium_pct, a.nitrogen_pct, a.oxygen_pct,
                a.carbon_dioxide_pct, a.methane_pct, a.water_vapor_pct
            FROM ATMOSPHERIC_ANALYSIS a
            JOIN EXOPLANETS e ON a.planet_id = e.planet_id
            WHERE e.name = '{planet_name}'
        '''
    else:
        # Get top 4 planets with highest biosignature potential
        query = '''
            SELECT
                e.name,
                a.hydrogen_pct, a.helium_pct, a.nitrogen_pct, a.oxygen_pct,
                a.carbon_dioxide_pct, a.methane_pct, a.water_vapor_pct
            FROM ATMOSPHERIC_ANALYSIS a
            JOIN EXOPLANETS e ON a.planet_id = e.planet_id
            JOIN HABITABILITY_SCORES h ON e.planet_id = h.planet_id
            WHERE h.habitability_class LIKE '%Class I%' OR h.habitability_class LIKE '%Class II%'
            ORDER BY a.oxygen_pct DESC, a.water_vapor_pct DESC
            LIMIT 4
        '''

    df = pd.read_sql_query(query, conn)

    if len(df) == 0:
        print("No atmospheric data available for the specified criteria.")
        return None

    # Radar chart setup
    categories = ['H₂', 'He', 'N₂', 'O₂', 'CO₂', 'CH₄', 'H₂O']
    n_cats = len(categories)
    angles = [n / float(n_cats) * 2 * np.pi for n in range(n_cats)]
    angles += angles[:1]  # Complete the loop

    fig, axes = plt.subplots(2, 2, figsize=(12, 12), subplot_kw=dict(polar=True))
    axes = axes.flatten()

    colors = ['#00ff88', '#ff6688', '#66aaff', '#ffaa00']

    for idx, (_, row) in enumerate(df.iterrows()):
        if idx >= 4:
            break

        ax = axes[idx]
        values = [row['hydrogen_pct'], row['helium_pct'], row['nitrogen_pct'],
                 row['oxygen_pct'], row['carbon_dioxide_pct'],
                 row['methane_pct'], row['water_vapor_pct']]
        values += values[:1]

        ax.plot(angles, values, 'o-', linewidth=2, color=colors[idx])
        ax.fill(angles, values, alpha=0.25, color=colors[idx])
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(categories, fontsize=10)
        ax.set_title(f"{row['name']}", fontsize=12, fontweight='bold', pad=20)
        ax.set_ylim(0, max(100, max(values) * 1.1))

    # Hide unused subplots
    for idx in range(len(df), 4):
        axes[idx].set_visible(False)

    fig.suptitle('Atmospheric Composition of Top Candidate Planets',
                fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

    return df


def plot_star_map(conn):
    """
    Creates a sky map showing locations of star systems with habitable planets.
    """
    query = '''
        SELECT
            s.name as star_name,
            s.ra_deg,
            s.dec_deg,
            s.distance_ly,
            s.spectral_type,
            COUNT(e.planet_id) as num_planets,
            MAX(h.esi_score) as best_esi,
            SUM(CASE WHEN h.hz_status = 'In HZ' THEN 1 ELSE 0 END) as hz_planets
        FROM STAR_SYSTEMS s
        LEFT JOIN EXOPLANETS e ON s.star_id = e.star_id
        LEFT JOIN HABITABILITY_SCORES h ON e.planet_id = h.planet_id
        GROUP BY s.star_id
        HAVING num_planets > 0
    '''
    df = pd.read_sql_query(query, conn)

    fig, ax = plt.subplots(figsize=(16, 8), subplot_kw={'projection': 'mollweide'})

    # Convert RA/Dec to radians for mollweide projection
    ra_rad = np.radians(df['ra_deg'] - 180)  # Center on 180°
    dec_rad = np.radians(df['dec_deg'])

    # Color by best ESI score
    scatter = ax.scatter(
        ra_rad, dec_rad,
        c=df['best_esi'],
        s=df['num_planets'] * 30 + 20,
        cmap='coolwarm',
        alpha=0.7,
        edgecolors='white',
        linewidth=0.5
    )

    # Highlight systems with HZ planets
    hz_systems = df[df['hz_planets'] > 0]
    hz_ra = np.radians(hz_systems['ra_deg'] - 180)
    hz_dec = np.radians(hz_systems['dec_deg'])
    ax.scatter(hz_ra, hz_dec, facecolors='none', edgecolors='#00ff00',
              s=150, linewidth=2, marker='*', label=f'With HZ Planets ({len(hz_systems)})')

    ax.set_title('Galactic Sky Map: Star Systems with Known Exoplanets\n(Size: # of Planets, Color: Best ESI)',
                fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.set_xlabel('Right Ascension', fontsize=11)
    ax.set_ylabel('Declination', fontsize=11)

    cbar = plt.colorbar(scatter, ax=ax, orientation='horizontal',
                        shrink=0.6, pad=0.1)
    cbar.set_label('Best ESI Score in System', fontsize=11)

    ax.legend(loc='lower right', fontsize=10)

    plt.tight_layout()
    plt.show()

    return df


print("✅ Visualization functions defined!")

# %%
# =============================================================================
# SECTION 7: INTERACTIVE DASHBOARD (ipywidgets)
# =============================================================================
# Build an interactive interface for exploring the database

def create_interactive_dashboard(conn):
    """
    Creates an interactive dashboard with multiple views and controls.
    """
    if not WIDGETS_AVAILABLE:
        print("⚠️ Interactive widgets not available.  Showing static visualizations instead.")
        print("\n📊 Generating static reports.. .\n")
        plot_habitability_distribution(conn)
        plot_stellar_habitability_heatmap(conn)
        plot_discovery_timeline(conn)
        plot_planet_scatter(conn)
        return

    # Create output areas
    main_output = widgets.Output()
    table_output = widgets.Output()

    # Style configuration
    style = {'description_width': '120px'}
    layout = widgets.Layout(width='300px')

    # =========================================================================
    # Tab 1: Planet Search Interface
    # =========================================================================

    # Search filters
    min_esi_slider = widgets.FloatSlider(
        value=0, min=0, max=1, step=0.05,
        description='Min ESI Score:',
        style=style, layout=layout
    )

    max_distance_slider = widgets.FloatSlider(
        value=500, min=10, max=1000, step=10,
        description='Max Distance (ly):',
        style=style, layout=layout
    )

    planet_type_dropdown = widgets.Dropdown(
        options=['All', 'Terrestrial', 'Super-Earth', 'Mini-Neptune',
                 'Neptune', 'Jupiter', 'Hot'],
        value='All',
        description='Planet Type:',
        style=style, layout=layout
    )

    hz_only_checkbox = widgets.Checkbox(
        value=False,
        description='Habitable Zone Only',
        style=style
    )

    spectral_dropdown = widgets.Dropdown(
        options=['All', 'G', 'K', 'M', 'F', 'A'],
        value='All',
        description='Star Type:',
        style=style, layout=layout
    )

    search_button = widgets.Button(
        description='🔍 Search Planets',
        button_style='primary',
        layout=widgets.Layout(width='200px')
    )

    def on_search_click(b):
        with table_output:
            clear_output(wait=True)

            p_type = None if planet_type_dropdown.value == 'All' else planet_type_dropdown.value
            spec = None if spectral_dropdown.value == 'All' else spectral_dropdown.value

            results = search_planets(
                conn,
                min_esi=min_esi_slider.value if min_esi_slider.value > 0 else None,
                max_distance=max_distance_slider.value,
                planet_type=p_type,
                hz_only=hz_only_checkbox.value,
                spectral_type=spec
            )

            print(f"🔎 Found {len(results)} planets matching your criteria:\n")
            display(results.style.background_gradient(subset=['esi'], cmap='RdYlGn')
                   .format({'esi': '{:.4f}', 'water_prob': '{:.1f}%'}))

    search_button.on_click(on_search_click)

    search_controls = widgets.VBox([
        widgets.HTML("<h3>🔭 Exoplanet Search</h3>"),
        min_esi_slider,
        max_distance_slider,
        planet_type_dropdown,
        spectral_dropdown,
        hz_only_checkbox,
        search_button
    ])

    search_tab = widgets.HBox([search_controls, table_output])

    # =========================================================================
    # Tab 2: Visualizations
    # =========================================================================

    viz_dropdown = widgets.Dropdown(
        options=[
            ('Habitability Distribution', 'hab_dist'),
            ('Stellar Type Analysis', 'stellar'),
            ('Discovery Timeline', 'timeline'),
            ('Mass-Radius Diagram', 'scatter'),
            ('Star Map', 'map'),
            ('Atmospheric Composition', 'atm')
        ],
        value='hab_dist',
        description='Visualization:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='350px')
    )

    viz_button = widgets.Button(
        description='📊 Generate Chart',
        button_style='success',
        layout=widgets.Layout(width='200px')
    )

    viz_output = widgets.Output()

    def on_viz_click(b):
        with viz_output:
            clear_output(wait=True)

            if viz_dropdown.value == 'hab_dist':
                plot_habitability_distribution(conn)
            elif viz_dropdown.value == 'stellar':
                plot_stellar_habitability_heatmap(conn)
            elif viz_dropdown.value == 'timeline':
                plot_discovery_timeline(conn)
            elif viz_dropdown.value == 'scatter':
                plot_planet_scatter(conn)
            elif viz_dropdown.value == 'map':
                plot_star_map(conn)
            elif viz_dropdown.value == 'atm':
                plot_atmospheric_composition(conn)

    viz_button.on_click(on_viz_click)

    viz_tab = widgets.VBox([
        widgets.HTML("<h3>📊 Data Visualizations</h3>"),
        widgets.HBox([viz_dropdown, viz_button]),
        viz_output
    ])

    # =========================================================================
    # Tab 3: Reports
    # =========================================================================

    report_dropdown = widgets.Dropdown(
        options=[
            ('Top Habitable Candidates', 'top'),
            ('Mission Performance', 'missions'),
            ('Biosignature Detections', 'biosig'),
            ('Planet Type Distribution', 'types'),
            ('Habitability Summary', 'summary')
        ],
        value='top',
        description='Report Type:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='350px')
    )

    report_button = widgets.Button(
        description='📋 Generate Report',
        button_style='info',
        layout=widgets.Layout(width='200px')
    )

    report_output = widgets.Output()

    def on_report_click(b):
        with report_output:
            clear_output(wait=True)

            if report_dropdown.value == 'top':
                print("🌍 TOP 20 HABITABLE PLANET CANDIDATES\n" + "="*60)
                df = get_top_candidates(conn)
                display(df.style.background_gradient(subset=['esi_score'], cmap='RdYlGn'))

            elif report_dropdown.value == 'missions':
                print("🚀 DISCOVERY MISSION PERFORMANCE RANKING\n" + "="*60)
                df = get_mission_performance_ranking(conn)
                display(df)

            elif report_dropdown.value == 'biosig':
                print("🧬 PLANETS WITH BIOSIGNATURE POTENTIAL\n" + "="*60)
                df = get_atmospheric_biosignatures(conn)
                display(df.style.background_gradient(subset=['biosig_potential_score'], cmap='YlGn'))

            elif report_dropdown.value == 'types':
                print("🪐 PLANET TYPE DISTRIBUTION\n" + "="*60)
                df = get_planet_type_distribution(conn)
                display(df)

            elif report_dropdown.value == 'summary':
                print("📊 HABITABILITY CLASSIFICATION SUMMARY\n" + "="*60)
                df = get_habitability_summary(conn)
                display(df)

    report_button.on_click(on_report_click)

    report_tab = widgets.VBox([
        widgets.HTML("<h3>📋 Analysis Reports</h3>"),
        widgets.HBox([report_dropdown, report_button]),
        report_output
    ])

    # =========================================================================
    # Tab 4: Custom SQL Query
    # =========================================================================

    sql_textarea = widgets.Textarea(
        value='''-- Enter your custom SQL query here
-- Example: Find all Earth-like planets within 50 light-years
SELECT
    e.name as planet,
    s.name as star,
    ROUND(s.distance_ly, 1) as distance_ly,
    ROUND(h.esi_score, 4) as esi
FROM EXOPLANETS e
JOIN STAR_SYSTEMS s ON e.star_id = s.star_id
JOIN HABITABILITY_SCORES h ON e.planet_id = h.planet_id
WHERE s.distance_ly < 50 AND h.esi_score > 0.7
ORDER BY h.esi_score DESC
LIMIT 20;''',
        layout=widgets.Layout(width='100%', height='200px')
    )

    sql_button = widgets.Button(
        description='▶️ Execute Query',
        button_style='warning',
        layout=widgets.Layout(width='200px')
    )

    sql_output = widgets.Output()

    def on_sql_click(b):
        with sql_output:
            clear_output(wait=True)
            try:
                query = sql_textarea.value.strip()
                if query.upper().startswith('SELECT'):
                    df = pd.read_sql_query(query, conn)
                    print(f"✅ Query returned {len(df)} rows:\n")
                    display(df)
                else:
                    print("⚠️ Only SELECT queries are allowed in this interface.")
            except Exception as e:
                print(f"❌ Query Error: {e}")

    sql_button.on_click(on_sql_click)

    sql_tab = widgets.VBox([
        widgets.HTML("<h3>💻 Custom SQL Query</h3>"),
        widgets.HTML("<p><i>Tables: STAR_SYSTEMS, EXOPLANETS, DISCOVERY_MISSIONS, PLANET_DISCOVERIES, ATMOSPHERIC_ANALYSIS, HABITABILITY_SCORES</i></p>"),
        sql_textarea,
        sql_button,
        sql_output
    ])

    # =========================================================================
    # Combine into Tab Interface
    # =========================================================================

    tabs = widgets.Tab(children=[search_tab, viz_tab, report_tab, sql_tab])
    tabs.set_title(0, '🔍 Search')
    tabs.set_title(1, '📊 Visualize')
    tabs.set_title(2, '📋 Reports')
    tabs.set_title(3, '💻 SQL')

    # Header
    header = widgets.HTML('''
        <div style="background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
                    padding: 20px; border-radius: 10px; margin-bottom: 20px;">
            <h1 style="color: #00ff88; margin: 0;">🌌 GEDHAS</h1>
            <h3 style="color: #66aaff; margin: 5px 0;">Galactic Exoplanet Discovery & Habitability Assessment System</h3>
            <p style="color: #aaaaaa; margin: 0;">Interactive Database Dashboard for Exoplanet Analysis</p>
        </div>
    ''')

    # Display the dashboard
    display(header)
    display(tabs)

    print("\n✅ Interactive Dashboard loaded!  Use the tabs above to explore the data.")


print("✅ Interactive dashboard function defined!")

# %%
# =============================================================================
# SECTION 8: MAIN EXECUTION
# =============================================================================
# Initialize database, populate with data, and launch dashboard

print("\n" + "="*70)
print("🚀 LAUNCHING GEDHAS - GALACTIC EXOPLANET DISCOVERY SYSTEM")
print("="*70 + "\n")

# Step 1: Create the database schema
conn = create_database()

# Step 2: Populate with mock data
stats = populate_database(conn)

# Step 3: Display database statistics
print("\n📈 DATABASE STATISTICS")
print("="*40)

cursor = conn.cursor()

# Count records in each table
for table in ['STAR_SYSTEMS', 'EXOPLANETS', 'DISCOVERY_MISSIONS',
              'PLANET_DISCOVERIES', 'ATMOSPHERIC_ANALYSIS', 'HABITABILITY_SCORES']:
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    count = cursor.fetchone()[0]
    print(f"   {table}: {count:,} records")

print("="*40)

# %%
# =============================================================================
# SECTION 9: QUICK DATA PREVIEW
# =============================================================================
# Show sample data from each table before launching dashboard

print("\n📋 SAMPLE DATA PREVIEW")
print("="*70)

print("\n🌟 STAR_SYSTEMS (First 5 rows):")
display(pd.read_sql_query("SELECT * FROM STAR_SYSTEMS LIMIT 5", conn))

print("\n🪐 EXOPLANETS (First 5 rows):")
display(pd.read_sql_query("SELECT * FROM EXOPLANETS LIMIT 5", conn))

print("\n🔭 DISCOVERY_MISSIONS:")
display(pd.read_sql_query("SELECT * FROM DISCOVERY_MISSIONS", conn))

print("\n🌍 TOP HABITABILITY CANDIDATES:")
display(get_top_candidates(conn, limit=10))

# %%
# =============================================================================
# SECTION 10: LAUNCH INTERACTIVE DASHBOARD
# =============================================================================

print("\n" + "="*70)
print("🎛️  LAUNCHING INTERACTIVE DASHBOARD")
print("="*70 + "\n")

create_interactive_dashboard(conn)

# %%
# =============================================================================
# SECTION 11: EXAMPLE QUERIES FOR REFERENCE
# =============================================================================
# Demonstrating various SQL techniques used in the application

print("\n" + "="*70)
print("📚 SQL QUERY EXAMPLES FOR REFERENCE")
print("="*70)

# Example 1: Complex JOIN with aggregation
print("\n1️⃣ Multi-table JOIN with Aggregation:")
print("-" * 50)
query1 = '''
SELECT
    s.spectral_type,
    COUNT(DISTINCT e.planet_id) as planet_count,
    ROUND(AVG(h.esi_score), 4) as avg_esi,
    SUM(CASE WHEN h.hz_status = 'In HZ' THEN 1 ELSE 0 END) as hz_count
FROM STAR_SYSTEMS s
JOIN EXOPLANETS e ON s.star_id = e.star_id
JOIN HABITABILITY_SCORES h ON e.planet_id = h.planet_id
GROUP BY s.spectral_type
ORDER BY avg_esi DESC
LIMIT 7
'''
print(query1)
display(pd.read_sql_query(query1, conn))

# Example 2: Window function
print("\n2️⃣ Window Function (Ranking):")
print("-" * 50)
query2 = '''
SELECT
    name,
    planet_type,
    esi_score,
    RANK() OVER (PARTITION BY planet_type ORDER BY esi_score DESC) as rank_in_type
FROM (
    SELECT e.name, e.planet_type, h.esi_score
    FROM EXOPLANETS e
    JOIN HABITABILITY_SCORES h ON e.planet_id = h.planet_id
)
WHERE rank_in_type <= 2
ORDER BY planet_type, rank_in_type
'''
# Note: SQLite has limited window function support, simplified version:
query2_simple = '''
SELECT
    e.name,
    e.planet_type,
    ROUND(h.esi_score, 4) as esi_score,
    h.habitability_class
FROM EXOPLANETS e
JOIN HABITABILITY_SCORES h ON e.planet_id = h.planet_id
WHERE h.habitability_class LIKE '%Class I%'
ORDER BY h.esi_score DESC
LIMIT 10
'''
print(query2_simple)
display(pd.read_sql_query(query2_simple, conn))

# Example 3: Subquery with EXISTS
print("\n3️⃣ Subquery with EXISTS (Stars with Biosignature Candidates):")
print("-" * 50)
query3 = '''
SELECT DISTINCT
    s.name as star_name,
    s.spectral_type,
    s.distance_ly
FROM STAR_SYSTEMS s
WHERE EXISTS (
    SELECT 1
    FROM EXOPLANETS e
    JOIN ATMOSPHERIC_ANALYSIS a ON e.planet_id = a.planet_id
    WHERE e.star_id = s.star_id
    AND a.biosignature_detected = 1
)
ORDER BY s.distance_ly
'''
print(query3)
display(pd.read_sql_query(query3, conn))

# Example 4: Complex CASE statement
print("\n4️⃣ Complex CASE Statement (Planet Classification):")
print("-" * 50)
query4 = '''
SELECT
    e.name,
    e.mass_earth,
    e.radius_earth,
    e.equilibrium_temp_k,
    CASE
        WHEN e.mass_earth < 2 AND e.equilibrium_temp_k BETWEEN 200 AND 350 THEN '🌍 Earth-like'
        WHEN e.mass_earth < 10 AND e.equilibrium_temp_k BETWEEN 200 AND 350 THEN '🌎 Super-Earth'
        WHEN e.mass_earth > 100 AND e.equilibrium_temp_k > 1000 THEN '🔥 Hot Jupiter'
        WHEN e.equilibrium_temp_k < 200 THEN '❄️ Frozen World'
        ELSE '🪐 Other'
    END as classification
FROM EXOPLANETS e
JOIN HABITABILITY_SCORES h ON e.planet_id = h.planet_id
WHERE h.esi_score > 0.5
ORDER BY e.equilibrium_temp_k
LIMIT 15
'''
print(query4)
display(pd.read_sql_query(query4, conn))

# %%
# =============================================================================
# SECTION 12: CLEANUP & SUMMARY
# =============================================================================

print("\n" + "="*70)
print("📊 FINAL SUMMARY")
print("="*70)

# Final statistics
summary_query = '''
SELECT
    (SELECT COUNT(*) FROM STAR_SYSTEMS) as total_stars,
    (SELECT COUNT(*) FROM EXOPLANETS) as total_planets,
    (SELECT COUNT(*) FROM EXOPLANETS WHERE confirmed = 1) as confirmed_planets,
    (SELECT COUNT(*) FROM HABITABILITY_SCORES WHERE habitability_class LIKE '%Class I%') as prime_candidates,
    (SELECT COUNT(*) FROM HABITABILITY_SCORES WHERE hz_status = 'In HZ') as in_habitable_zone,
    (SELECT COUNT(*) FROM ATMOSPHERIC_ANALYSIS WHERE biosignature_detected = 1) as biosignatures,
    (SELECT ROUND(MAX(esi_score), 4) FROM HABITABILITY_SCORES) as highest_esi,
    (SELECT ROUND(AVG(esi_score), 4) FROM HABITABILITY_SCORES) as avg_esi
'''
summary = pd.read_sql_query(summary_query, conn)

print(f"""
🌟 GEDHAS Database Summary:
{'='*40}
Total Star Systems:       {summary['total_stars'].values[0]:,}
Total Exoplanets:         {summary['total_planets'].values[0]:,}
Confirmed Planets:        {summary['confirmed_planets'].values[0]:,}
Prime Candidates:         {summary['prime_candidates'].values[0]:,}
In Habitable Zone:        {summary['in_habitable_zone'].values[0]:,}
Biosignatures Detected:   {summary['biosignatures'].values[0]:,}
Highest ESI Score:        {summary['highest_esi'].values[0]:.4f}
Average ESI Score:        {summary['avg_esi'].values[0]:.4f}
{'='*40}

🎓 DATABASE CONCEPTS DEMONSTRATED:
- DDL: CREATE TABLE with constraints, indexes
- DML: INSERT with batch generation
- Complex JOINs: INNER, LEFT, multi-table
- Aggregations: COUNT, SUM, AVG, MIN, MAX
- GROUP BY with HAVING
- Subqueries: Scalar, EXISTS, IN
- Window Functions: RANK, cumulative sums
- CASE expressions for conditional logic
- Date functions: strftime
- Parameterized queries for security
- Index optimization

🔬 ASTRONOMICAL CONCEPTS IMPLEMENTED:
- Habitable Zone calculations
- Earth Similarity Index (ESI)
- Equilibrium temperature physics
- Spectral type classification
- Atmospheric composition analysis
- Biosignature detection criteria

✅ Notebook execution complete!
""")

# Close connection when done (optional - keeps it open for further exploration)
# conn.close()


✅ All imports successful!
   - Interactive widgets: Available
   - Plotly charts: Available
✅ Astronomical calculation functions defined!
✅ Advanced SQL query functions defined!
✅ Visualization functions defined!
✅ Interactive dashboard function defined!

🚀 LAUNCHING GEDHAS - GALACTIC EXOPLANET DISCOVERY SYSTEM

🔧 Creating GEDHAS Database Schema...
   ✓ Created STAR_SYSTEMS table
   ✓ Created EXOPLANETS table
   ✓ Created DISCOVERY_MISSIONS table
   ✓ Created PLANET_DISCOVERIES table
   ✓ Created ATMOSPHERIC_ANALYSIS table
   ✓ Created HABITABILITY_SCORES table
   ✓ Created performance indexes
✅ Database schema created successfully!

📊 BATCH DATA GENERATION STARTING
🌟 Generating 150 star systems...
   ✓ Created 150 star systems
🪐 Generating exoplanets for 150 star systems...
   ✓ Created 441 exoplanets
🚀 Generating discovery missions...
   ✓ Created 12 discovery missions
🔭 Generating discovery records...
   ✓ Created 861 discovery records
🌫️ Generating atmospheric analyses...
   ✓ Crea

,star_id,name,spectral_type,luminosity,temperature,distance_ly,age_gyr,mass_solar,ra_deg,dec_deg,galactic_quadrant,metallicity
0,1,Delta Andromeda 759,K0,0.2073,4263,232.73,2.04,0.499,151.8919,-84.6365,Alpha,0.070
1,2,Alpha Leonis 826,K8,0.1946,4559,27.39,6.07,0.656,100.0337,-51.2435,Gamma,0.349
2,3,Iota Sideris 748,M6,0.0294,2598,71.51,2.50,0.207,332.0263,-75.8160,Gamma,-0.113
3,4,Gamma Nebulae 888,K1,0.4239,3793,20.65,4.34,0.721,228.8464,-24.3302,Gamma,0.407
4,5,Theta Cosmicus 474,M4,0.0491,2546,57.72,12.87,0.143,230.3999,10.2510,Alpha,-0.230



🪐 EXOPLANETS (First 5 rows):


,planet_id,star_id,name,mass_earth,radius_earth,orbital_period_days,semi_major_axis_au,eccentricity,equilibrium_temp_k,surface_gravity_earth,density_gcc,planet_type,discovery_date,confirmed
0,1,1,Delta Andromeda 759 b,4.045,1.457,2176.41,2.6070,0.1837,88.2,1.905,7.188,Cold Super-Earth,2005-07-01,1
1,2,1,Delta Andromeda 759 c,6.162,1.971,216.19,0.5592,0.0023,190.4,1.586,4.427,Cold Super-Earth,1996-02-13,1
2,3,1,Delta Andromeda 759 d,5.796,2.028,3350.26,3.4756,0.1675,76.4,1.409,3.821,Cold Super-Earth,2000-08-07,1
3,4,1,Delta Andromeda 759 e,0.943,1.037,33615.28,16.1683,0.0995,35.4,0.878,4.657,Cold Terrestrial,2002-07-26,1
4,5,2,Alpha Leonis 826 b,2.431,1.556,4268.44,4.4746,0.2703,80.3,1.005,3.552,Cold Super-Earth,2007-04-10,1



🔭 DISCOVERY_MISSIONS:


...


3️⃣ Subquery with EXISTS (Stars with Biosignature Candidates):
--------------------------------------------------

SELECT DISTINCT
    s.name as star_name,
    s.spectral_type,
    s.distance_ly
FROM STAR_SYSTEMS s
WHERE EXISTS (
    SELECT 1
    FROM EXOPLANETS e
    JOIN ATMOSPHERIC_ANALYSIS a ON e.planet_id = a.planet_id
    WHERE e.star_id = s.star_id
    AND a.biosignature_detected = 1
)
ORDER BY s.distance_ly



,star_name,spectral_type,distance_ly



4️⃣ Complex CASE Statement (Planet Classification):
--------------------------------------------------

SELECT
    e.name,
    e.mass_earth,
    e.radius_earth,
    e.equilibrium_temp_k,
    CASE
        WHEN e.mass_earth < 2 AND e.equilibrium_temp_k BETWEEN 200 AND 350 THEN '🌍 Earth-like'
        WHEN e.mass_earth < 10 AND e.equilibrium_temp_k BETWEEN 200 AND 350 THEN '🌎 Super-Earth'
        WHEN e.mass_earth > 100 AND e.equilibrium_temp_k > 1000 THEN '🔥 Hot Jupiter'
        WHEN e.equilibrium_temp_k < 200 THEN '❄️ Frozen World'
        ELSE '🪐 Other'
    END as classification
FROM EXOPLANETS e
JOIN HABITABILITY_SCORES h ON e.planet_id = h.planet_id
WHERE h.esi_score > 0.5
ORDER BY e.equilibrium_temp_k
LIMIT 15



,name,mass_earth,radius_earth,equilibrium_temp_k,classification
0,Alpha Stellara 16 b,1.286,0.966,133.2,❄️ Frozen World
1,Sigma Galaxia 372 d,0.846,1.001,136.6,❄️ Frozen World
2,Epsilon Lyrae 674 b,1.917,1.081,150.0,❄️ Frozen World
3,Rho Pegasi 277 c,2.266,1.383,155.9,❄️ Frozen World
4,Kappa Orionis 324 c,2.808,1.342,161.4,❄️ Frozen World
5,Lambda Lyrae 43 c,1.627,1.083,169.2,❄️ Frozen World
6,Tau Nebulae 739 c,0.982,1.007,170.4,❄️ Frozen World
7,Kappa Coronae 384 f,1.557,1.147,174.3,❄️ Frozen World
8,Delta Solaris 22 f,0.321,0.661,174.7,❄️ Frozen World
9,Tau Nebulae 739 b,2.215,1.306,182.1,❄️ Frozen World



📊 FINAL SUMMARY

🌟 GEDHAS Database Summary:
Total Star Systems:       150
Total Exoplanets:         441
Confirmed Planets:        423
Prime Candidates:         309
In Habitable Zone:        105
Biosignatures Detected:   0
Highest ESI Score:        0.9091
Average ESI Score:        0.3187

🎓 DATABASE CONCEPTS DEMONSTRATED:
- DDL: CREATE TABLE with constraints, indexes
- DML: INSERT with batch generation
- Complex JOINs: INNER, LEFT, multi-table
- Aggregations: COUNT, SUM, AVG, MIN, MAX
- GROUP BY with HAVING
- Subqueries: Scalar, EXISTS, IN
- Window Functions: RANK, cumulative sums
- CASE expressions for conditional logic
- Date functions: strftime
- Parameterized queries for security
- Index optimization

🔬 ASTRONOMICAL CONCEPTS IMPLEMENTED:
- Habitable Zone calculations
- Earth Similarity Index (ESI)
- Equilibrium temperature physics
- Spectral type classification
- Atmospheric composition analysis
- Biosignature detection criteria

✅ Notebook execution complete!

